In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%cd /home/vagrawal/RVT-CO2/code

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader
from torchsummary import summary
from models import *
from datasets import resp_co2_dataset
import seaborn as sns

from train_utils import Trainer
import utils

In [ ]:
%matplotlib notebook
plt.rcParams["figure.figsize"] = [9,4]

In [ ]:
%matplotlib inline
plt.rcParams["figure.figsize"] = [16,4]

In [ ]:
def stats_bean_plot(corr_stats):
    corr_stats_copy = corr_stats.copy()    
    del corr_stats_copy['mape_co2'], corr_stats_copy['z_co2'], corr_stats_copy['z_petco2']
    df_stats = pd.DataFrame.from_dict(corr_stats_copy)
    sns.violinplot(data = pd.DataFrame.from_dict(corr_stats_copy))

In [ ]:
dict_stats = {"split_and_model_name": [],
                "mean_corr_co2" : [],
                "mean_corr_petco2" : [],
                "mean_z_co2" : [],
                "mean_z_petco2" : [],
                "mean_mse_co2" : [],
                "mean_mse_petco2" : [],
                "mean_mae_co2" : [],
                "mean_mae_petco2" : [],
                "mean_mape_co2" : [],
                "mean_mape_petco2" : [],
                "std_corr_co2" : [],
                "std_corr_petco2" : [],
                "std_z_co2" : [],
                "std_z_petco2" : [],
                "std_mse_co2" : [],
                "std_mse_petco2" : [],
                "std_mae_co2" : [],
                "std_mae_petco2" : [],
                "std_mape_co2" : [],
                "std_mape_petco2" : [],}

def add_mean_std_stats(current_model_name, corr_stats):
    current_split_and_model_name = str(data_split_number) + '-' + current_model_name
    if current_split_and_model_name not in dict_stats[f"split_and_model_name"]:
        dict_stats["split_and_model_name"].append(current_split_and_model_name) 
        for key, value in corr_stats.items():
            dict_stats[f'mean_{key}'].append(round(np.mean(value), 3)) 
            dict_stats[f'std_{key}'].append(round(np.std(value), 3))

# dataloader

In [ ]:
baycrest_root = '/home/vagrawal/RVT-CO2/data/raw_physio_backup-biopac-20180417/preprocessed'
data_split_number = 1
txt_root = os.path.join(baycrest_root, 'splits_train_test', str(data_split_number))
apply_co2_norm = True

In [ ]:
dataloader_train = resp_co2_dataset(txt_root, resp_filename = 'resp_train.txt', co2_filename = 'co2_train.txt',apply_co2_norm=apply_co2_norm)
# dataloader_val = resp_co2_dataset(txt_root, resp_filename = 'resp_val.txt', co2_filename = 'co2_val.txt',apply_co2_norm=apply_co2_norm)
dataloader_test = resp_co2_dataset(txt_root, resp_filename = 'resp_test.txt', co2_filename = 'co2_test.txt',apply_co2_norm=apply_co2_norm)

dataloader_all = resp_co2_dataset(os.path.join(baycrest_root, 'splits_train_test')
                                  , resp_filename = 'resp_all_filenames.txt', co2_filename = 'co2_all_filenames.txt',apply_co2_norm=apply_co2_norm)

dataloaders = {
    'train':DataLoader(dataloader_train, batch_size=1, shuffle=True),
#     'val':DataLoader(dataloader_val, batch_size=1, shuffle=True),
    'test':DataLoader(dataloader_test, batch_size=1, shuffle=False),
    'all':DataLoader(dataloader_all, batch_size=1, shuffle=False)
}

# Save model paths

In [ ]:
# Config
train_models = False
save_models = False
save_logs = False
load_models = True
split_number = data_split_number ### the spilt for model is same as split for data
phases_train = ['train', 'test']
phases_test = ['test']

date = '20201011_baycrest'
save_root = os.path.join('./saved_models', date)
save_log_root = os.path.join('./saved_logs', date)

# Linear Regression without tanh

In [ ]:
current_model_name = 'linear_without_tanh'
current_model = linear_without_tanh(in_channels=1, out_channels=1)
save_dir = os.path.join(save_root, current_model_name)
save_path = os.path.join(save_dir, f'{split_number}.pth.gz')

In [ ]:
optimizer_lr = 0.01
net = Trainer(current_model, dataloaders, optimizer_lr)
print(net.nnArchitecture)
if load_models:
    net.load_model(save_path)

In [ ]:
if train_models:
    num_epochs = 15
    loss = net.nnTrain(num_epochs, lossFunction=torch.nn.MSELoss(), phases=phases_train, plot_after_epoch=(False, 2))

In [ ]:
add_mean_std_stats(current_model_name, corr_stats)

In [ ]:
if save_logs:
    save_log_dir = os.path.join(save_log_root, current_model_name)
    save_log_path = os.path.join(save_log_dir, f'{split_number}.npy')
    os.makedirs(save_log_dir, exist_ok=True)
    np.save(save_log_path,corr_stats)

In [ ]:
if save_models:
    os.makedirs(save_dir, exist_ok=True)
    net.save_model(save_path)

# Linear Regression with tanh

In [ ]:
current_model_name = 'linear_with_tanh'
current_model = linear_with_tanh(in_channels=1, out_channels=1)
save_dir = os.path.join(save_root, current_model_name)
save_path = os.path.join(save_dir, f'{split_number}.pth.gz')

In [ ]:
optimizer_lr = 0.01
net = Trainer(current_model, dataloaders, optimizer_lr)
print(net.nnArchitecture)
if load_models:
    net.load_model(save_path)

In [ ]:
if train_models:
    num_epochs = 15
    loss = net.nnTrain(num_epochs, lossFunction=torch.nn.MSELoss(), phases=phases_train, plot_after_epoch=(False, 2))

In [ ]:
add_mean_std_stats(current_model_name, corr_stats)

In [ ]:
if save_logs:
    save_log_dir = os.path.join(save_log_root, current_model_name)
    save_log_path = os.path.join(save_log_dir, f'{split_number}.npy')
    os.makedirs(save_log_dir, exist_ok=True)
    np.save(save_log_path,corr_stats)

In [ ]:
if save_models:
    os.makedirs(save_dir, exist_ok=True)
    net.save_model(save_path)

# For 1 layer NN

In [ ]:
current_model_name = 'conv_1_layer'
current_model = conv_1_layer(in_channels=1, out_channels=1)
save_dir = os.path.join(save_root, current_model_name)
save_path = os.path.join(save_dir, f'{split_number}.pth.gz')

In [ ]:
optimizer_lr = 0.01
net = Trainer(current_model, dataloaders, optimizer_lr)
print(net.nnArchitecture)
if load_models:
    net.load_model(save_path)

In [ ]:
if train_models:
    num_epochs = 15
    loss = net.nnTrain(num_epochs, lossFunction=torch.nn.MSELoss(), phases=phases_train, plot_after_epoch=(False, 2))

In [ ]:
add_mean_std_stats(current_model_name, corr_stats)

In [ ]:
if save_logs:
    save_log_dir = os.path.join(save_log_root, current_model_name)
    save_log_path = os.path.join(save_log_dir, f'{split_number}.npy')
    os.makedirs(save_log_dir, exist_ok=True)
    np.save(save_log_path,corr_stats)

In [ ]:
if save_models:
    os.makedirs(save_dir, exist_ok=True)
    net.save_model(save_path)

# For 2 layer NN

In [ ]:
current_model_name = 'conv_2_layer'
current_model = conv_2_layer(in_channels=1, out_channels=1)
save_dir = os.path.join(save_root, current_model_name)
save_path = os.path.join(save_dir, f'{split_number}.pth.gz')

In [ ]:
optimizer_lr = 0.01
net = Trainer(current_model, dataloaders, optimizer_lr)
summary(net.nnArchitecture, (1,800))
if load_models:
    net.load_model(save_path)

In [ ]:
if train_models:
    num_epochs = 15
    loss = net.nnTrain(num_epochs, lossFunction=torch.nn.MSELoss(), phases=phases_train, plot_after_epoch=(False, 2))

In [ ]:
add_mean_std_stats(current_model_name, corr_stats)

In [ ]:
if save_logs:
    save_log_dir = os.path.join(save_log_root, current_model_name)
    save_log_path = os.path.join(save_log_dir, f'{split_number}.npy')
    os.makedirs(save_log_dir, exist_ok=True)
    np.save(save_log_path,corr_stats)

In [ ]:
if save_models:
    os.makedirs(save_dir, exist_ok=True)
    net.save_model(save_path)

# For 4 layer NN

In [ ]:
current_model_name = 'conv_4_layer'
current_model = conv_4_layer(in_channels=1, out_channels=1)
save_dir = os.path.join(save_root, current_model_name)
save_path = os.path.join(save_dir, f'{split_number}.pth.gz')

In [ ]:
optimizer_lr = 0.01
net = Trainer(current_model, dataloaders, optimizer_lr)
summary(net.nnArchitecture, (1,800))
if load_models:
    net.load_model(save_path)

In [ ]:
if train_models:
    num_epochs = 15
    loss = net.nnTrain(num_epochs, lossFunction=torch.nn.MSELoss(), phases=phases_train, plot_after_epoch=(False, 2))

In [ ]:
add_mean_std_stats(current_model_name, corr_stats)

In [ ]:
if save_logs:
    save_log_dir = os.path.join(save_log_root, current_model_name)
    save_log_path = os.path.join(save_log_dir, f'{split_number}.npy')
    os.makedirs(save_log_dir, exist_ok=True)
    np.save(save_log_path,corr_stats)

In [ ]:
if save_models:
    os.makedirs(save_dir, exist_ok=True)
    net.save_model(save_path)

# For 6 layes NN

In [ ]:
current_model_name = 'conv_6_layer'
current_model = conv_6_layer(in_channels=1, out_channels=1)
save_dir = os.path.join(save_root, current_model_name)
save_path = os.path.join(save_dir, f'{split_number}.pth.gz')

In [ ]:
optimizer_lr = 0.005
net = Trainer(current_model, dataloaders, optimizer_lr)
summary(net.nnArchitecture, (1,3000))
if load_models:
    net.load_model(save_path)

In [ ]:
if train_models:
    num_epochs = 15
    loss = net.nnTrain(num_epochs, lossFunction=torch.nn.MSELoss(), phases=phases_train, plot_after_epoch=(False, 2))

In [ ]:
add_mean_std_stats(current_model_name, corr_stats)

In [ ]:
if save_logs:
    save_log_dir = os.path.join(save_log_root, current_model_name)
    save_log_path = os.path.join(save_log_dir, f'{split_number}.npy')
    os.makedirs(save_log_dir, exist_ok=True)
    np.save(save_log_path,corr_stats)

In [ ]:
if save_models:
    os.makedirs(save_dir, exist_ok=True)
    net.save_model(save_path)

# Weighted Loss Function

## For 1 layer NN with weighted loss function

In [ ]:
current_model_name = 'conv_1_layer_weightedLoss'
current_model = conv_1_layer(in_channels=1, out_channels=1)
save_dir = os.path.join(save_root, current_model_name)
save_path = os.path.join(save_dir, f'{split_number}.pth.gz')

In [ ]:
optimizer_lr = 0.01
net = Trainer(current_model, dataloaders, optimizer_lr)
print(net.nnArchitecture)
if load_models:
    net.load_model(save_path)

In [ ]:
if train_models:
    num_epochs = 15
    loss = net.nnTrain_peaks(num_epochs, lossFunction=torch.nn.MSELoss(), phases=phases_train, plot_after_epoch=(False, 2))

In [ ]:
add_mean_std_stats(current_model_name, corr_stats)

In [ ]:
if save_logs:
    save_log_dir = os.path.join(save_log_root, current_model_name)
    save_log_path = os.path.join(save_log_dir, f'{split_number}.npy')
    os.makedirs(save_log_dir, exist_ok=True)
    np.save(save_log_path,corr_stats)

In [ ]:
if save_models:
    os.makedirs(save_dir, exist_ok=True)
    net.save_model(save_path)

## For 2 layer NN with weighted loss function

In [ ]:
current_model_name = 'conv_2_layer_weightedLoss'
current_model = conv_2_layer(in_channels=1, out_channels=1)
save_dir = os.path.join(save_root, current_model_name)
save_path = os.path.join(save_dir, f'{split_number}.pth.gz')

In [ ]:
optimizer_lr = 0.01
net = Trainer(current_model, dataloaders, optimizer_lr)
summary(net.nnArchitecture, (1,800))
if load_models:
    net.load_model(save_path)

In [ ]:
if train_models:
    num_epochs = 15
    loss = net.nnTrain_peaks(num_epochs, lossFunction=torch.nn.MSELoss(), phases=phases_train, plot_after_epoch=(False, 2))

In [ ]:
add_mean_std_stats(current_model_name, corr_stats)

In [ ]:
if save_logs:
    save_log_dir = os.path.join(save_log_root, current_model_name)
    save_log_path = os.path.join(save_log_dir, f'{split_number}.npy')
    os.makedirs(save_log_dir, exist_ok=True)
    np.save(save_log_path,corr_stats)

In [ ]:
if save_models:
    os.makedirs(save_dir, exist_ok=True)
    net.save_model(save_path)

## For 4 layer NN with weighted loss function

In [ ]:
current_model_name = 'conv_4_layer_weightedLoss'
current_model = conv_4_layer(in_channels=1, out_channels=1)
save_dir = os.path.join(save_root, current_model_name)
save_path = os.path.join(save_dir, f'{split_number}.pth.gz')

In [ ]:
optimizer_lr = 0.005
net = Trainer(current_model, dataloaders, optimizer_lr)
# summary(net.nnArchitecture, (1,3000))
if load_models:
    net.load_model(save_path)

In [ ]:
if train_models:
    num_epochs = 1
    loss = net.nnTrain_peaks(num_epochs, lossFunction=torch.nn.MSELoss(), phases=phases_train, plot_after_epoch=(False, 2))

In [ ]:
add_mean_std_stats(current_model_name, corr_stats)

In [ ]:
if save_logs:
    save_log_dir = os.path.join(save_log_root, current_model_name)
    save_log_path = os.path.join(save_log_dir, f'{split_number}.npy')
    os.makedirs(save_log_dir, exist_ok=True)
    np.save(save_log_path,corr_stats)

In [ ]:
if save_models:
    os.makedirs(save_dir, exist_ok=True)
    net.save_model(save_path)

## Weighted 6 layers NN

In [ ]:
current_model_name = 'conv_6_layer_weightedLoss'
current_model = conv_6_layer(in_channels=1, out_channels=1)
save_dir = os.path.join(save_root, current_model_name)
save_path = os.path.join(save_dir, f'{split_number}.pth.gz')

In [ ]:
optimizer_lr = 0.005
net = Trainer(current_model, dataloaders, optimizer_lr)
if load_models:
    net.load_model(save_path)

In [ ]:
if train_models:
    num_epochs = 15
    loss = net.nnTrain_peaks(num_epochs, lossFunction=torch.nn.MSELoss(), phases=phases_train, plot_after_epoch=(False, 2))

In [ ]:
add_mean_std_stats(current_model_name, corr_stats)

In [ ]:
if save_logs:
    save_log_dir = os.path.join(save_log_root, current_model_name)
    save_log_path = os.path.join(save_log_dir, f'{split_number}.npy')
    os.makedirs(save_log_dir, exist_ok=True)
    np.save(save_log_path,corr_stats)

In [ ]:
if save_models:
    os.makedirs(save_dir, exist_ok=True)
    net.save_model(save_path)

# Combine Stats

In [ ]:
df_stats = pd.DataFrame(dict_stats)

In [ ]:
df_stats